# Molecular Music: Sonifying Protein Binding Data

This notebook implements a comprehensive sonification system for molecular biology that translates protein binding characteristics into musical parameters.

## Overview
- **Binding Affinity (Kd)** → **Pitch/Frequency**
- **Binding Kinetics** → **Temporal Parameters**
- **Protein Structure** → **Timbral Characteristics**
- **Molecular Weight** → **Amplitude/Volume**
- **Binding Cooperativity** → **Harmonic Relationships**

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pythonosc import udp_client
from pythonosc.osc_message_builder import OscMessageBuilder
import requests
import json
import time
import warnings
warnings.filterwarnings("ignore")

print("Libraries imported successfully!")

## 1. Data Access Functions

We'll implement functions to access molecular data from public databases:
- **ChEMBL**: Bioactivity database with binding affinities
- **BindingDB**: Database of measured binding affinities  
- **PDB**: 3D protein structures
- **UniProt**: Protein sequences and functional annotations

In [ ]:
class MolecularDataAccess:
    """Class for accessing molecular data from public databases"""
    
    def __init__(self):
        self.chembl_base_url = "https://www.ebi.ac.uk/chembl/api/data"
        self.bindingdb_base_url = "https://www.bindingdb.org/axis2/services/BDBService"
        
    def get_chembl_activities(self, target_chembl_id, limit=100):
        """
        Fetch binding activities from ChEMBL for a specific target
        
        Args:
            target_chembl_id (str): ChEMBL target ID (e.g., 'CHEMBL279')
            limit (int): Maximum number of activities to retrieve
            
        Returns:
            pandas.DataFrame: Binding activity data
        """
        try:
            url = f"{self.chembl_base_url}/activity.json"
            params = {
                'target_chembl_id': target_chembl_id,
                'limit': limit,
                'format': 'json'
            }
            
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            data = response.json()
            activities = data.get('activities', [])
            
            # Convert to DataFrame
            df = pd.DataFrame(activities)
            
            # Filter for binding data
            binding_types = ['Kd', 'Ki', 'IC50', 'EC50']
            if 'standard_type' in df.columns:
                df = df[df['standard_type'].isin(binding_types)]
            
            return df
            
        except Exception as e:
            print(f"Error fetching ChEMBL data: {e}")
            return pd.DataFrame()
    
    def get_sample_binding_data(self):
        """
        Generate sample binding data for demonstration purposes
        """
        np.random.seed(42)
        n_compounds = 50
        
        data = {
            'compound_id': [f'COMP_{i:03d}' for i in range(n_compounds)],
            'kd_nm': np.random.lognormal(mean=3, sigma=2, size=n_compounds),  # Kd in nM
            'kon_m1s1': np.random.lognormal(mean=12, sigma=1, size=n_compounds),  # kon in M-1s-1
            'koff_s1': np.random.lognormal(mean=-2, sigma=1, size=n_compounds),  # koff in s-1
            'molecular_weight': np.random.normal(400, 150, n_compounds),  # MW in Da
            'structure_type': np.random.choice(['alpha_helix', 'beta_sheet', 'random_coil'], n_compounds),
            'cooperativity': np.random.choice(['positive', 'negative', 'none'], n_compounds, p=[0.2, 0.1, 0.7])
        }
        
        df = pd.DataFrame(data)
        df['kd_nm'] = np.clip(df['kd_nm'], 0.1, 10000)  # Realistic Kd range
        df['molecular_weight'] = np.clip(df['molecular_weight'], 100, 1000)  # Realistic MW range
        
        return df

# Initialize data access
data_access = MolecularDataAccess()
print("Molecular data access initialized!")

## 2. Parameter Mapping Functions

These functions translate molecular parameters into musical parameters according to the mappings defined in the README.

In [ ]:
class MolecularToMusicalMapper:
    """Class for mapping molecular parameters to musical parameters"""
    
    def __init__(self):
        # Musical parameter ranges
        self.freq_range = (50, 2000)  # Hz
        self.attack_range = (0.01, 2.0)  # seconds
        self.release_range = (0.1, 5.0)  # seconds
        self.amplitude_range = (0.1, 1.0)  # normalized
        
    def kd_to_frequency(self, kd_values):
        """
        Map Kd values to frequencies
        Low Kd (strong binding) → Lower frequencies (stable, foundational)
        High Kd (weak binding) → Higher frequencies (unstable, transient)
        
        Args:
            kd_values (array-like): Kd values in nM
            
        Returns:
            array: Frequencies in Hz
        """
        kd_array = np.array(kd_values)
        
        # Log scale mapping (Kd spans orders of magnitude)
        log_kd = np.log10(np.clip(kd_array, 0.1, 10000))
        log_kd_norm = (log_kd - np.log10(0.1)) / (np.log10(10000) - np.log10(0.1))
        
        # Invert so low Kd = low frequency
        freq_norm = 1 - log_kd_norm
        
        # Map to frequency range
        frequencies = self.freq_range[0] + freq_norm * (self.freq_range[1] - self.freq_range[0])
        
        return frequencies
    
    def kinetics_to_temporal(self, kon_values, koff_values):
        """
        Map binding kinetics to temporal parameters
        kon (association rate) → Attack time (how quickly sound begins)
        koff (dissociation rate) → Release time (how quickly sound fades)
        
        Args:
            kon_values (array-like): Association rates in M-1s-1
            koff_values (array-like): Dissociation rates in s-1
            
        Returns:
            tuple: (attack_times, release_times) in seconds
        """
        kon_array = np.array(kon_values)
        koff_array = np.array(koff_values)
        
        # Log scale normalization
        log_kon = np.log10(np.clip(kon_array, 1e3, 1e8))
        log_koff = np.log10(np.clip(koff_array, 1e-4, 1e2))
        
        # Normalize to [0, 1]
        kon_norm = (log_kon - 3) / (8 - 3)  # 1e3 to 1e8 range
        koff_norm = (log_koff - (-4)) / (2 - (-4))  # 1e-4 to 1e2 range
        
        # Invert: fast kinetics = short times
        attack_norm = 1 - kon_norm
        release_norm = 1 - koff_norm
        
        # Map to time ranges
        attack_times = self.attack_range[0] + attack_norm * (self.attack_range[1] - self.attack_range[0])
        release_times = self.release_range[0] + release_norm * (self.release_range[1] - self.release_range[0])
        
        return attack_times, release_times
    
    def structure_to_waveform(self, structure_types):
        """
        Map protein structure to waveform types
        α-helices → Sine waves (smooth, periodic structure)
        β-sheets → Sawtooth waves (structured, angular geometry)  
        Random coils → Noise (chaotic, unstructured regions)
        
        Args:
            structure_types (array-like): Structure type strings
            
        Returns:
            array: Waveform type indices (0=sine, 1=sawtooth, 2=noise)
        """
        structure_map = {
            'alpha_helix': 0,
            'beta_sheet': 1, 
            'random_coil': 2
        }
        
        return np.array([structure_map.get(s, 0) for s in structure_types])
    
    def molecular_weight_to_amplitude(self, mw_values):
        """
        Map molecular weight to amplitude
        Larger proteins → Higher amplitude (more molecular mass = more presence)
        
        Args:
            mw_values (array-like): Molecular weights in Da
            
        Returns:
            array: Amplitude values (normalized)
        """
        mw_array = np.array(mw_values)
        
        # Normalize molecular weight
        mw_norm = (mw_array - 100) / (1000 - 100)  # 100-1000 Da range
        mw_norm = np.clip(mw_norm, 0, 1)
        
        # Map to amplitude range
        amplitudes = self.amplitude_range[0] + mw_norm * (self.amplitude_range[1] - self.amplitude_range[0])
        
        return amplitudes
    
    def cooperativity_to_harmony(self, cooperativity_types):
        """
        Map binding cooperativity to harmonic relationships
        Positive cooperativity → Consonant harmonies (perfect 5ths, octaves)
        Negative cooperativity → Dissonant intervals (minor 2nds, tritones)
        No cooperativity → Unison or simple intervals
        
        Args:
            cooperativity_types (array-like): Cooperativity type strings
            
        Returns:
            array: Harmonic interval ratios
        """
        harmony_map = {
            'positive': [1.0, 1.5, 2.0],  # Unison, perfect 5th, octave
            'negative': [1.0, 1.059, 1.414],  # Unison, minor 2nd, tritone
            'none': [1.0, 1.125, 1.25]  # Unison, major 2nd, minor 3rd
        }
        
        harmonies = []
        for coop_type in cooperativity_types:
            harmonies.append(harmony_map.get(coop_type, harmony_map['none']))
        
        return harmonies

# Initialize mapper
mapper = MolecularToMusicalMapper()
print("Molecular to musical mapper initialized!")

## 3. SuperCollider Integration

This section handles communication with SuperCollider (sclang) via OSC (Open Sound Control) messages.

In [ ]:
class SuperColliderInterface:
    """Interface for communicating with SuperCollider via OSC"""
    
    def __init__(self, ip="127.0.0.1", port=57120):
        """
        Initialize OSC client for SuperCollider communication
        
        Args:
            ip (str): SuperCollider server IP address
            port (int): SuperCollider server port
        """
        self.client = udp_client.SimpleUDPClient(ip, port)
        self.ip = ip
        self.port = port
        
    def send_protein_binding_event(self, kd, kon, koff, structure, mw, amp=0.3):
        """
        Send protein binding event to SuperCollider
        
        Args:
            kd (float): Dissociation constant in nM
            kon (float): Association rate in M-1s-1
            koff (float): Dissociation rate in s-1
            structure (int): Structure type (0=sine, 1=sawtooth, 2=noise)
            mw (float): Molecular weight in Da
            amp (float): Amplitude (0-1)
        """
        try:
            self.client.send_message("/protein/bind", [kd, kon, koff, structure, mw, amp])
            print(f"Sent protein binding event: Kd={kd:.2f}nM, structure={structure}")
        except Exception as e:
            print(f"Error sending OSC message: {e}")
    
    def send_cooperative_binding_event(self, sites, kd1, kd2, cooperativity, amp=0.2):
        """
        Send cooperative binding event to SuperCollider
        
        Args:
            sites (int): Number of binding sites
            kd1 (float): First site Kd in nM
            kd2 (float): Second site Kd in nM
            cooperativity (float): Cooperativity factor
            amp (float): Amplitude (0-1)
        """
        try:
            self.client.send_message("/protein/cooperative", [sites, kd1, kd2, cooperativity, amp])
            print(f"Sent cooperative binding event: {sites} sites, cooperativity={cooperativity}")
        except Exception as e:
            print(f"Error sending OSC message: {e}")
    
    def send_enzyme_kinetics_event(self, vmax, km, substrate, amp=0.3):
        """
        Send enzyme kinetics event to SuperCollider
        
        Args:
            vmax (float): Maximum velocity
            km (float): Michaelis constant
            substrate (float): Substrate concentration
            amp (float): Amplitude (0-1)
        """
        try:
            self.client.send_message("/enzyme/kinetics", [vmax, km, substrate, amp])
            print(f"Sent enzyme kinetics event: Vmax={vmax}, Km={km}, [S]={substrate}")
        except Exception as e:
            print(f"Error sending OSC message: {e}")
    
    def generate_supercollider_code(self):
        """
        Generate SuperCollider code for the sonification SynthDefs
        
        Returns:
            str: SuperCollider code
        """
        sc_code = '''
// Molecular Music Sonification SynthDefs
// Copy and paste this code into SuperCollider

// Start the audio server
s.boot;

// Core protein binding SynthDef
SynthDef(\\proteinBinding, { |out=0, kd=1000, kon=0.1, koff=0.05, 
                           structure=0, mw=50000, amp=0.3|
    
    // Map binding affinity to frequency (log scale)
    var freq = kd.linexp(1, 10000, 80, 800);
    
    // Map kinetics to envelope
    var attack = kon.linexp(1000, 100000000, 0.01, 2);   // kon → attack time
    var release = koff.linexp(0.0001, 100, 0.1, 5);      // koff → release time
    
    // Map molecular weight to amplitude
    var dynAmp = mw.linlin(100, 1000, 0.1, 1.0) * amp;
    
    // Map structure to waveform
    var sig = SelectX.ar(structure, [
        SinOsc.ar(freq),           // α-helix (smooth)
        Saw.ar(freq) * 0.7,        // β-sheet (structured)
        WhiteNoise.ar * 0.3        // random coil (chaotic)
    ]);
    
    var env = Env.perc(attack, release).kr(doneAction:2);
    Out.ar(out, sig * env * dynAmp ! 2);
}).add;

// Cooperative binding SynthDef
SynthDef(\\cooperativeBinding, { |out=0, sites=4, kd1=100, kd2=50, 
                               cooperativity=2, amp=0.2|
    var freqs = Array.fill(sites, { |i| 
        var effective_kd = kd1 * (kd2/kd1).pow(i * cooperativity);
        effective_kd.linexp(10, 1000, 200, 800);
    });
    
    var sig = Mix(freqs.collect({ |freq, i|
        var delay = i * 0.1; // Staggered binding
        var env = Env.perc(0.01, 1).kr(doneAction:0);
        DelayN.ar(SinOsc.ar(freq) * env, 1, delay);
    }));
    
    Out.ar(out, sig * amp ! 2);
}).add;

// Enzyme kinetics SynthDef  
SynthDef(\\enzymeKinetics, { |out=0, vmax=1, km=50, substrate=100, amp=0.3|
    // Michaelis-Menten equation: v = vmax * [S] / (km + [S])
    var velocity = vmax * substrate / (km + substrate);
    var freq = velocity.linlin(0, 1, 100, 400);
    var rate = velocity.linlin(0, 1, 0.5, 8); // Turnover rate
    
    var sig = SinOsc.ar(freq) * Env.perc(0.01, 1/rate).kr(doneAction:2);
    Out.ar(out, sig * amp ! 2);
}).add;

// OSC responders for real-time data
OSCdef(\\bindingEvent, { |msg|
    var kd = msg[1];
    var kon = msg[2]; 
    var koff = msg[3];
    var structure = msg[4];
    var mw = msg[5];
    var amp = msg[6];
    
    Synth(\\proteinBinding, [
        \\kd, kd, \\kon, kon, \\koff, koff, 
        \\structure, structure, \\mw, mw, \\amp, amp
    ]);
}, '/protein/bind');

OSCdef(\\cooperativeEvent, { |msg|
    var sites = msg[1];
    var kd1 = msg[2];
    var kd2 = msg[3];
    var coop = msg[4];
    var amp = msg[5];
    
    Synth(\\cooperativeBinding, [
        \\sites, sites, \\kd1, kd1, \\kd2, kd2, \\cooperativity, coop, \\amp, amp
    ]);
}, '/protein/cooperative');

OSCdef(\\enzymeEvent, { |msg|
    var vmax = msg[1];
    var km = msg[2];
    var substrate = msg[3];
    var amp = msg[4];
    
    Synth(\\enzymeKinetics, [
        \\vmax, vmax, \\km, km, \\substrate, substrate, \\amp, amp
    ]);
}, '/enzyme/kinetics');

"SuperCollider SynthDefs loaded and OSC responders active!".postln;
'''
        return sc_code

# Initialize SuperCollider interface
sc_interface = SuperColliderInterface()
print("SuperCollider interface initialized!")
print("\\nTo use with SuperCollider:")
print("1. Start SuperCollider (sclang)")
print("2. Copy and paste the generated code")
print("3. Run the sonification functions below")

## 4. Phase 1: Proof of Concept Implementation

Let's implement a simple protein-ligand interaction sonification as described in the README.

In [ ]:
# Generate sample binding data
binding_data = data_access.get_sample_binding_data()
print("Generated sample binding data:")
print(f"Number of compounds: {len(binding_data)}")
print("\\nFirst 5 compounds:")
display(binding_data.head())

# Basic statistics
print("\\nData statistics:")
print(f"Kd range: {binding_data['kd_nm'].min():.2f} - {binding_data['kd_nm'].max():.2f} nM")
print(f"MW range: {binding_data['molecular_weight'].min():.1f} - {binding_data['molecular_weight'].max():.1f} Da")
print(f"Structure types: {binding_data['structure_type'].value_counts().to_dict()}")
print(f"Cooperativity types: {binding_data['cooperativity'].value_counts().to_dict()}")